<a href="https://colab.research.google.com/github/prnishtala/NLP/blob/main/FinalProject/END_Phase_1_CapstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries


In [ ]:
import re
import spacy
import torch
import torchtext
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy.data import Field, BucketIterator, TabularDataset, Dataset, Example
import random
import math
import time
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex
import torch.nn.functional as F
import keyword, token, tokenize
import spacy
import torchtext
from torchtext import data

SEED = 1234
torch.manual_seed(SEED)
torch.set_deterministic(True)
np.random.seed(SEED)

/usr/local/lib/python3.7/dist-packages/torch/__init__.py:422: UserWarning: torch.set_deterministic is deprecated and will be removed in a future release. Please use torch.use_deterministic_algorithms instead
  "torch.set_deterministic is deprecated and will be removed in a future "


# Data Cleansing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! cp "/content/drive/My Drive/NLP/english_python_data_raw.txt" english_python_data.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read the file and remove the unwanted data
# Removed few anomalies manually to prevent over-engineering

with open('english_python_data.txt', 'r') as f:
  raw_data = f.readlines()

cleansed_data = []

for sentence in raw_data:
  # Remove the comments indicating beginning of Driver Codes
  if re.search(r'^\s*#\s+driver', sentence.lower()):
    continue
  # Remove the unwanted empty lines
  elif re.search(r'^\s*\n+\s*$', sentence.lower()):
    continue
  # Remove the comments in the pattern of "In[0-9]"
  elif re.search(r'^\s*#\s+in\[[0-9]+\]', sentence.lower()):
    continue
  # Remove the comments containing just the numbers
  elif re.search(r'^\s*#+\s*[0-9]*\s*\n', sentence.lower()):
    continue
  else:
    cleansed_data.append(sentence)


In [ ]:
# Generate DataFrame from the Input Data

df = pd.DataFrame(columns = ['src', 'trg'])

src = ''
trg = ''
minimum_length = 30

for i,record in enumerate(cleansed_data):
  if record.startswith('#') and len(record) > minimum_length:
    df.loc[len(df)] = [src,trg]
    trg = ''
    src = record
  else:
    trg = trg + record

df = df.iloc[1:]
df = df.reset_index(drop=True)
 

In [ ]:
file = open('english_python_data.txt',"rt", encoding='latin')
data_complete = file.read()
data_complete =  data_complete.lower()

data_complete = re.sub(r'"#[0123456789]','#', data_complete)
data_complete = data_complete.replace("\n\n","\n")
data_complete = data_complete.replace("\n\n\n","\n\n")

data_com = data_complete.split('#')

src = []
trg =[]

for each_code in data_com:
    pr_text = each_code.split('\n')
    src.append(pr_text[0])
    trg.append(pr_text[1:])

df = pd.DataFrame({'src': src, 'trg': trg})

# write a python program to add two numbers
num1 = 1.5
num2 = 6.3
sum = num1 + num2
print(f'Sum: {sum}')


# write a python function to add two user provided numbers and return the sum
def add_two_numbers(num1, num2):
    sum = num1 + num2
    return sum


# write a program to find and print the largest among three numbers

num1 = 10
num2 = 12
num3 = 14
if (num1 >= num2) and (num1 >= num3):
   largest = num1
elif (num2 >= num1) and (num2 >= num3):
   largest = num2
else:
   largest = num3
print(f'largest:{largest}')


# write a program to find and print the smallest among three numbers
num1 = 10
num2 = 12
num3 = 14
if (num1 <= num2) and (num1 <= num3):
   smallest = num1
elif (num2 <= num1) and (num2 <= num3):
   smallest = num2
else:
   smallest = num3
print(f'smallest:{smallest}')


# Wr
1103729


In [ ]:
df

,src,trg
0,,[]
1,write a python program to add two numbers,"[num1 = 1.5, num2 = 6.3, sum = num1 + num2, pr..."
2,write a python function to add two user provi...,"[def add_two_numbers(num1, num2):, sum = n..."
3,write a program to find and print the largest...,"[num1 = 10, num2 = 12, num3 = 14, if (num1 >= ..."
4,write a program to find and print the smalles...,"[num1 = 10, num2 = 12, num3 = 14, if (num1 <= ..."
...,...,...
4990,write a program to print bit wise or of two n...,"[a = 60, b = 13, c = a | b, print(""or"", c), , ]"
4991,write a program to print bit wise xor of two ...,"[a = 60, b = 13, c = a ^ b, print(""xor"", c), , ]"
4992,write a program to calculate binary ones comp...,"[a = 60, c = ~a, print(""binary ones complement..."
4993,write a program to binary left shift a number,"[c = a << 2, print(""binary left shift"", c), , ]"


# Tokenizing

In [ ]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
spacy_en = spacy.load('en')

In [ ]:
def custom_tokenizer(nlp):
    infix_re = re.compile(r'''[.\,\?\:\;\...\‘\’\`\“\”\"\'\(\)\[\]\{\}\*\%\^\+\-\=\<\>\|\!(//)(\n)(\t)~]''')
    prefix_re = compile_prefix_regex(nlp.Defaults.prefixes)
    suffix_re = compile_suffix_regex(nlp.Defaults.suffixes)

    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer,
                                token_match=None)

spacy_que = spacy.load('en_core_web_sm')
spacy_ans = spacy.load('en_core_web_sm')
spacy_ans.tokenizer = custom_tokenizer(spacy_ans)

def tokenize_que(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    text = re.sub(r'#','',text).strip()
    return [tok.text for tok in spacy_que.tokenizer(text)]

def tokenize_ans(text):
    """
    Tokenizes Code text from a string into a list of strings
    """
    return [tok.text for tok in spacy_ans.tokenizer(text)]

In [ ]:
SRC = Field(tokenize = 'spacy', 
            eos_token = '<eos>',
            init_token = '<sos>', 
            lower = True,
            batch_first = True)

TRG = Field(tokenize = 'spacy', 
            eos_token = '<eos>',
            init_token = '<sos>', 
            lower = True,
            batch_first = True)

fields = [("src", SRC), ("trg", TRG)]

In [ ]:
fields = [("src", SRC), ("trg", TRG)]
example = [data.Example.fromlist([df.src[i],df.trg[i]], fields) for i in range(df.shape[0])] 
dataset = data.Dataset(example, fields)



(train_data, valid_data, test_data) = dataset.split(split_ratio=[0.80, 0.10, 0.10], random_state=random.seed(SEED))

In [ ]:
print(vars(train_data.examples[2]))

{'src': [' ', 'write', 'a', 'program', 'that', 'adds', 'the', 'square', 'of', 'two', 'numbers', 'and', 'prints', 'it'], 'trg': ['a = 32', 'b = 21', 'result = a**2 + b**2', 'print(result)', '']}


In [ ]:
SRC.build_vocab(train_data, min_freq = 1)
TRG.build_vocab(train_data, min_freq = 1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print("Program_text Vocab size", len(SRC.vocab))
print("Program_code Vocab size", len(TRG.vocab))

Program_text Vocab size 2314
Program_code Vocab size 11795


In [ ]:
BATCH_SIZE = 64

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, valid_data), 
     sort= False,
     batch_size = BATCH_SIZE,
     device = device)

In [ ]:
import spacy
spacy_en = spacy.load('en')
def Tokenize(sentence):
  sentence = str(sentence).replace('\n', '\t\t')
  return [tok.text for tok in spacy_en.tokenizer(sentence)]

In [ ]:
import gensim
w2v_dim = 256
w2v_min_count = 2
w2v_window = 3
target = []
for sent in df['trg'].values:
  sent_token = Tokenize(sent)
  #sent_token = tokenize_ans(sent)
  target.append(sent_token)
w2v_model = gensim.models.Word2Vec(target, size = w2v_dim, window = w2v_window, min_count = w2v_min_count)

In [ ]:
from tqdm import tqdm_notebook
word2vec_vectors = []
for token, idx in tqdm_notebook(TRG.vocab.stoi.items()):
  if token in w2v_model.wv.vocab.keys():
    word2vec_vectors.append(torch.FloatTensor(w2v_model[token]))
  else:
    word2vec_vectors.append(torch.zeros(w2v_dim))
TRG.vocab.set_vectors(TRG.vocab.stoi, word2vec_vectors, w2v_dim)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
TRG.vocab.vectors

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
w2v_model.save('embeddings.txt')

In [ ]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 255):
        super().__init__()

        self.device = device
        
        #self.tok_embedding = nn.Embedding.from_pretrained(torch.FloatTensor(Program_text.vocab.vectors))
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 pre_trained_emb,
                 max_length = 255):
        super().__init__()
        
        self.device = device
        
        #self.tok_embedding = nn.Embedding.from_pretrained(torch.FloatTensor(Program_code.vocab.vectors))
        self.pre_trained_emb = pre_trained_emb
        self.tok_embedding = nn.Embedding.from_pretrained(self.pre_trained_emb)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # query, key, value
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
pre_trained_emb = torch.FloatTensor(TRG.vocab.vectors)

enc = Encoder(INPUT_DIM,
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device, 
              pre_trained_emb)

SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim()>1 and not isinstance(m,nn.Embedding):
        nn.init.xavier_normal_(m.weight.data)

model.apply(initialize_weights);

In [ ]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
        if isinstance(criterion, nn.CTCLoss):
            output_lengths = output.shape[1]
            target_lengths = trg.shape[1]-1
            output = output.contiguous().permute(1,0,2)
            trg = trg[:,1:].contiguous()
            loss = criterion(output, trg, output_lengths, target_lengths)
        else:
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            loss = criterion(output, trg)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, metrics=None):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            if isinstance(criterion, nn.CTCLoss):
                output_lengths = torch.full(size=(output.shape[0],), fill_value = output.shape[1], dtype=torch.long)
                target_lengths = trg.shape[1]-1
                output = output.contiguous().permute(1,0,2)
                trg = trg[:,1:].contiguous()
                loss = criterion(output, trg, output_lengths, target_lengths)

            else:
                output = output.contiguous().view(-1, output_dim)
                trg = trg[:,1:].contiguous().view(-1)
                loss = criterion(output, trg)
            # compute bleu score
            #if metrics is not None:
                
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
model.apply(initialize_weights);

In [ ]:
LEARNING_RATE = 0.0005

optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
#criterion = nn.CTCLoss()

In [ ]:
import time
N_EPOCHS = 50
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 4s
	Train Loss: 7.944 | Train PPL: 2818.080
	 Val. Loss: 7.529 |  Val. PPL: 1861.021
Epoch: 02 | Time: 0m 3s
	Train Loss: 7.142 | Train PPL: 1264.526
	 Val. Loss: 7.552 |  Val. PPL: 1903.649
Epoch: 03 | Time: 0m 4s
	Train Loss: 6.928 | Train PPL: 1020.208
	 Val. Loss: 7.572 |  Val. PPL: 1943.589
Epoch: 04 | Time: 0m 3s
	Train Loss: 6.748 | Train PPL: 852.729
	 Val. Loss: 7.480 |  Val. PPL: 1771.403
Epoch: 05 | Time: 0m 4s
	Train Loss: 6.587 | Train PPL: 725.450
	 Val. Loss: 7.524 |  Val. PPL: 1851.035
Epoch: 06 | Time: 0m 3s
	Train Loss: 6.465 | Train PPL: 642.123
	 Val. Loss: 7.436 |  Val. PPL: 1696.643
Epoch: 07 | Time: 0m 3s
	Train Loss: 6.340 | Train PPL: 566.589
	 Val. Loss: 7.431 |  Val. PPL: 1686.903
Epoch: 08 | Time: 0m 4s
	Train Loss: 6.209 | Train PPL: 497.346
	 Val. Loss: 7.396 |  Val. PPL: 1629.466
Epoch: 09 | Time: 0m 3s
	Train Loss: 6.058 | Train PPL: 427.380
	 Val. Loss: 7.289 |  Val. PPL: 1464.261
Epoch: 10 | Time: 0m 3s
	Train Loss: 5.873 | Train P

In [ ]:
import spacy
def generate_code(sentence, src_field, trg_field, model, device, max_len = 50):
    
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [ ]:
Question_text = 'write a python program to prints common letters in two input strings\n '

print(Question_text )

translation, attention = generate_code(Question_text , SRC, TRG, model, device)

for i in range(len(translation)):
  print(translation[i])

write a python program to prints common letters in two input strings
 
s1='python'
s2='schoolofai'
a=list(set(s1)&set(s2))
print("the common letters are:")
for i in a:
    print(i)

<eos>


In [ ]:
import random

for i in range(25):
  print("++++++++++++++++++++++\n\n\nExample", i+1)
  inf = random.randint(1,2500)
  src = vars(train_data.examples[inf])['src']
  print(' '.join(src))
  translation, attention = generate_code(src , SRC, TRG, model, device)

  for i in range(len(translation)):
    print(end =" ")
    print(translation[i])

++++++++++++++++++++++


Example 1
   generate 3 random integers between 100 and 999 which is divisible by 5
 import random
 print("generating 3 random integer number between 100 and 999 divisible by 5")
 for num in range(3):
     print(random.randrange(100, 999, 5), end=', ')
 
 <eos>
++++++++++++++++++++++


Example 2
  write a python program to prints common letters in two input strings
 s1='python'
 s2='schoolofai'
 a=list(set(s1)&set(s2))
 print("the common letters are:")
 for i in a:
     print(i)
 
 <eos>
++++++++++++++++++++++


Example 3
  96 write a python function that takes a number and returns an array of the number duplicated n times
 
 def duplicate_array(num, n):
     num = [num] * n
     return num
 
 
 <eos>
++++++++++++++++++++++


Example 4
  write a python program to find area of circle
 pi = 3.14
 radius = float(6)
 area = pi * radius * radius
 circumference = 2 * pi * radius
 print(f'area of a circle {area}')
 print(f'circumference of a circle {circumference}')
 